## Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from vizdoom import *


from skimage import transform 
from skimage.color import rgb2gray 

from collections import deque 

import time 
import random 
import warnings

warnings.filterwarnings('ignore')

/home/raed/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Init Environment

In [2]:
def init_env():
    game = DoomGame()
    game.load_config("basic.cfg")
    game.set_doom_scenario_path("basic.wad")
    game.init()
    
    left = [1,0,0]
    shoot = [0,0,1]
    right = [0,1,0]
    
    possible_actions = [left, right, shoot]
    
    return game, possible_actions 

In [3]:
def test_env(test_eps = 10):
    """
    Perform random action and test env to ensure it works 
    """
    game = DoomGame()
    game.load_config("basic.cfg")
    game.set_doom_scenario_path("basic.wad")
    game.init()
    
    left = [1,0,0]
    shoot = [0,0,1]
    right = [0,1,0]
    
    possible_actions = [left, right, shoot]
    
    for i in range(test_eps):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice(possible_actions)
            print (action)
            
            reward = game.make_action(action)
            print ("\treward:", reward)
            
            time.sleep(0.02)
        
        #prints every episode 
        print ("Result:", game.get_total_reward())
        time.sleep(2)
    
    game.close()
    

In [4]:
#test_env()

In [5]:
#call init_env 
game, possible_actions = init_env()

## Preprocessing 

In [6]:
def preprocess_frame(frame):
    """
    Convert frames to grayscale, crop irrelevent parts, normalize pixels, 
    resize frame.
    
    Input is frame of size 210x160x3
    Returns frame of size 84x84x1 
    """
    #graying done by vizdoom
    #grayed = rgb2gray(frame)
    
    cropped = frame[30:-10, 30:-30]
    normed = cropped/255.0 
    resized = transform.resize(normed, [84,84])
    
    return resized #returns preprocessed frame 

stack_size = 4
def stack_frames(stacked_frames, frame, is_new_episode, stack_size=stack_size):
    """
    takes a frame/state and preprocesses it, 
    if same episode:
        adds to the stack state 
    else new episode: 
        creates stacked state 
    
    where the stacked state is 4 stacked states(frames)
    returns stacked state where axis=1 is for different frames 
    """
    frame = preprocess_frame(frame)
    
    if not is_new_episode:
        stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
    
    else: 
        #init deque 
        stacked_frames = deque([np.zeros((84,84), dtype=np.int) 
                                for i in range(stack_size)], maxlen=4)
        #new episode so same frame x4 
        for i in range(4): stacked_frames.append(frame)
        
        stacked_state = np.stack(stacked_frames, axis=2)

    return stacked_state, stacked_frames
  

In [7]:
#init frame stack
stacked_frames  =  deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
frame = np.zeros((84,84), dtype=np.int)

#test helper functions 
sample_state, stacked_frames = stack_frames(stacked_frames, frame, True)
print(np.shape(sample_state), np.shape(stacked_frames))

(84, 84, 4) (4, 84, 84)


## Hyper Params

In [27]:
#training params  
total_episodes = 500 #total num of episodes 
total_test_episodes = 10 #total num of episodes to test on 
max_steps = 100 #max num of steps per episode 
bs = 64

#network params 
lr = 0.0002 #learning rate 
state_size = [84,84,4]  #4 84x84 frames 
action_size = game.get_available_buttons_size() # 3 actions

#discount factor 
gamma = 0.95 

#exploration params 
epsilon = 1.0 #starting value for eps greedy (explore)
max_eps = 1.0  #max value for eps greey 
min_eps = 0.01 #min value for eps greedy 
decay_rate = 0.0001 #decay rate for eps 

#Recall Params
memory_size = 1000000
pretrained_len = bs #number of init memories 

#Training Mode 
training=True 

#Env should be rendered or not 
episode_render = True 

## DQN: Model 


In [9]:
class DQN():
    def __init__(self, state_size, action_size, learning_rate, name='DQN', training=training):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = learning_rate
        self.training = training
        with tf.variable_scope(name):
            #placeholders
            self.inputs = tf.placeholder(tf.float32,[None, *self.state_size])
            self.actions = tf.placeholder(tf.float32, [None, self.action_size])
            self.target_Q = tf.placeholder(tf.float32)
            
            #block 1 
            self.conv1 = tf.layers.conv2d(inputs=self.inputs, filters=32, 
                                          kernel_size=[8,8], 
                                          strides=(4,4), 
                                          padding='valid', 
                                          kernel_initializer= tf.contrib.layers.xavier_initializer_conv2d())
            self.bn1 = tf.layers.batch_normalization(self.conv1, epsilon=1e-5, training=self.training)
            self.elu1 = tf.nn.elu(self.bn1)
            
            #block 2
            self.conv2 = tf.layers.conv2d(inputs=self.elu1, filters=64, 
                                          kernel_size=[4,4], 
                                          strides=(2,2), 
                                          padding='valid', 
                                          kernel_initializer= tf.contrib.layers.xavier_initializer_conv2d())
            self.bn2 = tf.layers.batch_normalization(self.conv2, epsilon=1e-5, training=self.training)
            self.elu2 = tf.nn.elu(self.bn2)
            
            #block 3
            self.conv3 = tf.layers.conv2d(inputs=self.elu2, filters=128, 
                                          kernel_size=[4,4], 
                                          strides=(2,2), 
                                          padding='valid', 
                                          kernel_initializer= tf.contrib.layers.xavier_initializer_conv2d())
            self.bn3 = tf.layers.batch_normalization(self.conv3, epsilon=1e-5, training=self.training)
            self.elu3 = tf.nn.elu(self.bn3)
            
            #FC Block 
            self.flat = tf.layers.flatten(self.elu3)
            self.fc = tf.layers.dense(self.flat, units=512, activation=tf.nn.elu, 
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            self.out = tf.layers.dense(self.fc, units=self.action_size, activation=None, 
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            
            #Q-Value prediction 
            self.pred_Q = tf.reduce_sum(tf.multiply(self.out, self.actions), axis=1)
            
            #Loss Function 
            self.loss = tf.reduce_mean(tf.square(self.target_Q-self.pred_Q))
            
            #Optimizer 
            self.optim = tf.train.RMSPropOptimizer(learning_rate=self.lr).minimize(self.loss)
           

In [10]:
tf.reset_default_graph()

#init
num_actions = len(possible_actions)
DQNetwork = DQN(state_size, num_actions, lr)

## Experience Replay

In [11]:
#memory class creates and manages deque 

class Memory():
    def __init__(self, mem_limit):
        self.cache = deque(maxlen=mem_limit)
    
    def add(self, exp):
        self.cache.append(exp)
    
    def sample(self, sample_size):
        sampling_ind = np.random.choice(np.arange(len(self.cache)), size=sample_size, replace=False)
        return [self.cache[i] for i in sampling_ind]

In [12]:
#activate and populate memory 

memory = Memory(mem_limit=memory_size)

import pdb 

game.new_episode()

for i in range(pretrained_len):
    #pdb.set_trace()
    #initilize state for first step  
    if i == 0: 
        state = game.get_state().screen_buffer #init state 
        state, stacked_frames = stack_frames(stacked_frames, state, True)
    
    #take a random action
    action = random.choice(possible_actions)
    
    #reward from chosen action 
    reward = game.make_action(action)
    
    #check if done 
    done = game.is_episode_finished()
    
    if not done:
        #get next state 
        next_state = game.get_state().screen_buffer
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
        #add memory 
        memory.add((state, action, reward, next_state, done))
        #update state
        state = next_state
        
    else: #were dead
        #update next state 
        next_state = np.zeros((84,84), dtype=np.int)

        #add exp 
        memory.add((state, action, reward, next_state, done))

        #new episode and restart 
        game.new_episode()
        state = game.get_state().screen_buffer
        state, stacked_frames = stack_frames(stacked_frames, state, True)


## DQN: Training

In [13]:
#Function to predict action based on epsilon greedy 

def explore_exploit(epsilon, min_eps, decay_rate, decay_step, state, num_actions):
    #produce random number between 0 and 1 
    check = np.random.rand()
    explore_prob = min_eps + (epsilon-min_eps)*np.exp(-decay_rate*decay_step)
    
    #explore
    if explore_prob>check:
        #take random action 
        action = random.choice(possible_actions)  
        
    #exploit 
    else:
        #q vals predicted by network 
        Qs = sess.run(DQNetwork.out, feed_dict = {DQNetwork.inputs : state.reshape((1,*state.shape))})
        #choose action corresponding to best Q value 
        choice = np.argmax(Qs)
        action = possible_actions[int(choice)]
    return action, explore_prob


In [14]:
#for saving 
saver = tf.train.Saver()

In [15]:
#training loop
if training: 
    with tf.Session() as sess:
        #init vars and decay step
        sess.run(tf.global_variables_initializer())
        decay_step = 0 
        
        #rewards for each episode 
        episodic_rewards = [] 
        
        #init doom  
        game.init()
        
        for episode in range(total_episodes):
            #init steps for episode 
            step = 0 
            
            #init observation
            game.init()
            game.new_episode()
            state = game.get_state().screen_buffer
            state, stacked_frame = stack_frames(stacked_frames, state, True)
            
            #list for rewards collected in episode 
            rewards_list = []
            
            while step < max_steps:
                ################SAMPLING#######################    
                #increments 
                step += 1
                decay_step += 1
                #if step % 50 == 0: print (step)
                #choose action 
                action, explore_prob = explore_exploit(epsilon, min_eps, decay_rate, decay_step, state, num_actions)

                #take action                 
                reward = game.make_action(action)
                
                #check if game is done 
                done = game.is_episode_finished()
                
                rewards_list.append(reward) 
                
                #if were not dead 
                if not done:
                    #get next state
                    next_state = game.get_state().screen_buffer
                    
                    #store transition after converting to proper format 
                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                    #add memory 
                    memory.add((state, action, reward, next_state, done))
                    
                    #update state
                    state = next_state

                #if were dead     
                else: 
                    #update next state 
                    next_state = np.zeros((84,84), dtype=np.int)

                    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    
                    #add episodic rewards 
                    tot_reward = np.sum(rewards_list)
                    episodic_rewards.append((episode, tot_reward))
                    
                    #end episode 
                    step = max_steps
                    
                    #add exp 
                    memory.add((state, action, reward, next_state, done))
        
                ################Learning#######################    
                #extract minibatch values 
                mini_batch = memory.sample(bs)
                states_batch = np.array([sample[0] for sample in mini_batch])
                actions_batch = np.array([sample[1] for sample in mini_batch])
                rewards_batch = np.array([sample[2] for sample in mini_batch])
                next_states_batch = np.array([sample[3] for sample in mini_batch])
                dones_batch = np.array([sample[4] for sample in mini_batch])
                
                #target qs to be set below 
                targ_qs_batch = []
                
                #get predicted Q's for each next state 
                next_qs_batch = sess.run(DQNetwork.out, feed_dict={DQNetwork.inputs : next_states_batch})
                
                #determine if state is terminal and set value for target_q
                for i in range(bs):
                    final_state = dones_batch[i]
                    
                    if final_state:
                        targ_qs_batch.append(rewards_batch[i])
                    
                    else:   
                        target_q = rewards_batch[i] + gamma * np.max(next_qs_batch[i])
                        targ_qs_batch.append(target_q)
                    
                
                #convert to np array 
                target_q = np.array([i for i in targ_qs_batch])
                
                #determine loss 
                loss, _ = sess.run([DQNetwork.loss, DQNetwork.optim], feed_dict={DQNetwork.inputs: states_batch,
                                                                               DQNetwork.target_Q: targ_qs_batch,
                                                                               DQNetwork.actions: actions_batch})
                
                
                #perform gradient update 
                
            #episode summary 
            print ("*********************")
            print ("Episode:\t", episode)
            print ("Reward:\t", np.sum(rewards_list))
            print ("Loss:\t", loss)
            print ("Was Terminal?:\t", done)

            if episode % 5 == 0:
                save_path = saver.save(sess, "./models/model.ckpt")
                print("Model Saved at episode: ", episode)              

*********************
Episode:	 0
Reward:	 -125.0
Loss:	 0.93442965
Was Terminal?:	 False
Model Saved at episode:  0
*********************
Episode:	 1
Reward:	 -125.0
Loss:	 0.8770241
Was Terminal?:	 False
*********************
Episode:	 2
Reward:	 89.0
Loss:	 0.55901897
Was Terminal?:	 True
*********************
Episode:	 3
Reward:	 68.0
Loss:	 153.41006
Was Terminal?:	 True
*********************
Episode:	 4
Reward:	 -130.0
Loss:	 23.615196
Was Terminal?:	 False
*********************
Episode:	 5
Reward:	 91.0
Loss:	 15.830358
Was Terminal?:	 True
Model Saved at episode:  5
*********************
Episode:	 6
Reward:	 -125.0
Loss:	 3.9352164
Was Terminal?:	 False
*********************
Episode:	 7
Reward:	 -125.0
Loss:	 5.265579
Was Terminal?:	 False
*********************
Episode:	 8
Reward:	 87.0
Loss:	 54.230595
Was Terminal?:	 True
*********************
Episode:	 9
Reward:	 -130.0
Loss:	 3.2960236
Was Terminal?:	 False
*********************
Episode:	 10
Reward:	 -125.0
Loss:	 4.1839294

*********************
Episode:	 88
Reward:	 45.0
Loss:	 6.272789
Was Terminal?:	 True
*********************
Episode:	 89
Reward:	 -125.0
Loss:	 4.676659
Was Terminal?:	 False
*********************
Episode:	 90
Reward:	 64.0
Loss:	 4.4888396
Was Terminal?:	 True
Model Saved at episode:  90
*********************
Episode:	 91
Reward:	 19.0
Loss:	 10.02096
Was Terminal?:	 True
*********************
Episode:	 92
Reward:	 82.0
Loss:	 9.397436
Was Terminal?:	 True
*********************
Episode:	 93
Reward:	 49.0
Loss:	 8.630248
Was Terminal?:	 True
*********************
Episode:	 94
Reward:	 95.0
Loss:	 5.0180902
Was Terminal?:	 True
*********************
Episode:	 95
Reward:	 -130.0
Loss:	 7.970933
Was Terminal?:	 False
Model Saved at episode:  95
*********************
Episode:	 96
Reward:	 16.0
Loss:	 11.76259
Was Terminal?:	 True
*********************
Episode:	 97
Reward:	 69.0
Loss:	 7.7393007
Was Terminal?:	 True
*********************
Episode:	 98
Reward:	 43.0
Loss:	 6.1060724
Was Termi

*********************
Episode:	 176
Reward:	 75.0
Loss:	 12.20301
Was Terminal?:	 True
*********************
Episode:	 177
Reward:	 -5.0
Loss:	 5.5080523
Was Terminal?:	 True
*********************
Episode:	 178
Reward:	 74.0
Loss:	 9.825525
Was Terminal?:	 True
*********************
Episode:	 179
Reward:	 72.0
Loss:	 6.888454
Was Terminal?:	 True
*********************
Episode:	 180
Reward:	 42.0
Loss:	 7.8049517
Was Terminal?:	 True
Model Saved at episode:  180
*********************
Episode:	 181
Reward:	 -120.0
Loss:	 5.2430797
Was Terminal?:	 False
*********************
Episode:	 182
Reward:	 35.0
Loss:	 3.6714854
Was Terminal?:	 True
*********************
Episode:	 183
Reward:	 56.0
Loss:	 8.539335
Was Terminal?:	 True
*********************
Episode:	 184
Reward:	 92.0
Loss:	 4.1609454
Was Terminal?:	 True
*********************
Episode:	 185
Reward:	 34.0
Loss:	 9.071602
Was Terminal?:	 True
Model Saved at episode:  185
*********************
Episode:	 186
Reward:	 71.0
Loss:	 7.86704

*********************
Episode:	 264
Reward:	 63.0
Loss:	 5.209791
Was Terminal?:	 True
*********************
Episode:	 265
Reward:	 70.0
Loss:	 9.129034
Was Terminal?:	 True
Model Saved at episode:  265
*********************
Episode:	 266
Reward:	 40.0
Loss:	 6.5835257
Was Terminal?:	 True
*********************
Episode:	 267
Reward:	 94.0
Loss:	 7.9093256
Was Terminal?:	 True
*********************
Episode:	 268
Reward:	 45.0
Loss:	 6.32007
Was Terminal?:	 True
*********************
Episode:	 269
Reward:	 68.0
Loss:	 8.069833
Was Terminal?:	 True
*********************
Episode:	 270
Reward:	 48.0
Loss:	 11.741457
Was Terminal?:	 True
Model Saved at episode:  270
*********************
Episode:	 271
Reward:	 94.0
Loss:	 6.0555935
Was Terminal?:	 True
*********************
Episode:	 272
Reward:	 51.0
Loss:	 11.460759
Was Terminal?:	 True
*********************
Episode:	 273
Reward:	 -15.0
Loss:	 9.169064
Was Terminal?:	 True
*********************
Episode:	 274
Reward:	 89.0
Loss:	 8.231043
W

*********************
Episode:	 352
Reward:	 91.0
Loss:	 8.645686
Was Terminal?:	 True
*********************
Episode:	 353
Reward:	 88.0
Loss:	 6.669792
Was Terminal?:	 True
*********************
Episode:	 354
Reward:	 95.0
Loss:	 7.4213486
Was Terminal?:	 True
*********************
Episode:	 355
Reward:	 72.0
Loss:	 10.090168
Was Terminal?:	 True
Model Saved at episode:  355
*********************
Episode:	 356
Reward:	 95.0
Loss:	 5.011202
Was Terminal?:	 True
*********************
Episode:	 357
Reward:	 75.0
Loss:	 5.2492704
Was Terminal?:	 True
*********************
Episode:	 358
Reward:	 67.0
Loss:	 7.346737
Was Terminal?:	 True
*********************
Episode:	 359
Reward:	 84.0
Loss:	 5.194503
Was Terminal?:	 True
*********************
Episode:	 360
Reward:	 -115.0
Loss:	 14.905788
Was Terminal?:	 False
Model Saved at episode:  360
*********************
Episode:	 361
Reward:	 40.0
Loss:	 8.283341
Was Terminal?:	 True
*********************
Episode:	 362
Reward:	 -115.0
Loss:	 14.398

*********************
Episode:	 440
Reward:	 73.0
Loss:	 10.131065
Was Terminal?:	 True
Model Saved at episode:  440
*********************
Episode:	 441
Reward:	 61.0
Loss:	 6.751134
Was Terminal?:	 True
*********************
Episode:	 442
Reward:	 83.0
Loss:	 13.970741
Was Terminal?:	 True
*********************
Episode:	 443
Reward:	 95.0
Loss:	 18.10119
Was Terminal?:	 True
*********************
Episode:	 444
Reward:	 57.0
Loss:	 7.979479
Was Terminal?:	 True
*********************
Episode:	 445
Reward:	 75.0
Loss:	 16.02927
Was Terminal?:	 True
Model Saved at episode:  445
*********************
Episode:	 446
Reward:	 42.0
Loss:	 7.6240463
Was Terminal?:	 True
*********************
Episode:	 447
Reward:	 76.0
Loss:	 5.1810274
Was Terminal?:	 True
*********************
Episode:	 448
Reward:	 -120.0
Loss:	 9.038743
Was Terminal?:	 False
*********************
Episode:	 449
Reward:	 48.0
Loss:	 6.015636
Was Terminal?:	 True
*********************
Episode:	 450
Reward:	 80.0
Loss:	 10.10771

In [30]:
%pwd

'/home/raed/Projects/rl-implementations/DQN'

In [32]:
test_episodes = 50

with tf.Session() as sess:
    
    #setup env 
    game, possible_actions = init_env()
    
    #load the model 
    saver.restore(sess, "./models/model.ckpt")
    game.init()
    total_score = 0 

    #run for each episode  
    for episode in range(test_episodes):
             
        game.new_episode()
        state = game.get_state().screen_buffer
        state, stacked_frames = stack_frames(stacked_frames, state, True)
    
        while not game.is_episode_finished():
            #choose action corresponding to best q val 
            q_preds = sess.run(DQNetwork.out, feed_dict={DQNetwork.inputs:state.reshape((1,*state.shape))})            
            choice = np.argmax(q_preds)
            action = possible_actions[int(choice)]
            
            #take action
            game.make_action(action)
            done = game.is_episode_finished()
            score = game.get_total_reward()
            
            if done:  #if game is done  
                break 
            
            else:
                next_state = game.get_state().screen_buffer
                next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                state = next_state 
            
        print (""">>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: {}""".format(game.get_total_reward())) 
        total_score += score
    
                 
    game.close()
    print ("****************************")
    print ("Average Reward over {} episodes:\t".format(test_episodes), total_score/float(test_episodes))
    

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 83.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 68.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 55.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 47.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 77.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 52.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 95.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 85.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
                    Total Reward: 54.0
>>>>>>>>>>>>>>>>>>>> TESTING SUMMARY >>>>>>>>>>>>>>>>>>>>
       